In [1]:
import time
import numpy as np
import pandas as pd

import Case
from imp import reload
reload(Case)
from tqdm import tqdm

C:\Users\DAHS\AppData\Local\Temp\ipykernel_34084\1818478163.py:6: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload


In [4]:
def number(df, mode):
    
    if mode == 'mimic':
        print('number of subject :', len(df.drop_duplicates(subset=["subject_id"])))
        print('number of stay :', len(df.drop_duplicates(subset=["stay_id"])))
        
    else:
        print('number of subject :', len(df.drop_duplicates(subset=["uniquepid"])))
        print('number of stay :', len(df.drop_duplicates(subset=["patientunitstayid"])))

In [6]:
mimic_path = '/Users/DAHS/Desktop/ECP_CONT/ECP_SCL/Case Labeling/MIMIC-IV.csv.gz'
eicu_path = '/Users/DAHS/Desktop/ECP_CONT/ECP_SCL/Case Labeling/eICU.csv.gz'

mimic = pd.read_csv(mimic_path, compression = 'gzip')
eicu = pd.read_csv(eicu_path, compression = 'gzip')

In [7]:
number(mimic, 'mimic')
number(eicu, 'eicu')

number of subject : 9880
number of stay : 10289
number of subject : 9554
number of stay : 9980


In [ ]:
mimic_annoted = Case.annotation(mimic, 'mimic')

analysis_cohort_for_mimic = []

idx = mimic_annoted[(mimic_annoted['ABPd']==75.00)|(mimic_annoted['ABPs']==125.00)].index

mimic_annoted['MAP_fillna'].loc[idx]=1
mimic_annoted['ABPd_fillna'].loc[idx]=1
mimic_annoted['ABPs_fillna'].loc[idx]=1

for stayid in mimic_annoted.stay_id.unique():

    interest = mimic_annoted[mimic_annoted['stay_id']==stayid].head(9) # 입원 후 9시간 동안은 no circ 인 경우만 포함
    lactic = mimic_annoted[mimic_annoted['stay_id']==stayid].head(2) 

    if all(interest['Annotation']=='no_circ'):
        analysis_cohort_for_mimic.append(stayid)
    
    if any(lactic['Lactate']==1.1):

        stay_df = mimic_annoted[mimic_annoted['stay_id']==stayid]
        lact_idx = stay_df[stay_df['Lactate']==1.1].index
        mimic_annoted['Lactate_fillna'].loc[lact_idx]=1

print('num of retain stay id: ', len(analysis_cohort_for_mimic))

check_mimic = mimic_annoted[mimic_annoted['stay_id'].isin(analysis_cohort_for_mimic)].reset_index(drop=True)
mimic_labeled = optimized_shock_labeler(check_mimic, 'mimic')


eicu_annoted = Case.annotation(eicu, 'eicu')

analysis_cohort_for_eicu = []

idx = eicu_annoted[(eicu_annoted['ABPd']==75.00)|(eicu_annoted['ABPs']==125.00)].index

eicu_annoted['MAP_fillna'].loc[idx]=1
eicu_annoted['ABPd_fillna'].loc[idx]=1
eicu_annoted['ABPs_fillna'].loc[idx]=1

for stayid in eicu_annoted.patientunitstayid.unique():

    interest = eicu_annoted[eicu_annoted['patientunitstayid']==stayid].head(9)
    lactic = eicu_annoted[eicu_annoted['patientunitstayid']==stayid].head(2)


    if all(interest['Annotation']=='no_circ'):
        analysis_cohort_for_eicu.append(stayid)
    
    if any(lactic['Lactate']==1.1):

        stay_df = eicu_annoted[eicu_annoted['patientunitstayid']==stayid]
        lact_idx = stay_df[stay_df['Lactate']==1.1].index
        eicu_annoted['Lactate_fillna'].loc[lact_idx]=1

print('num of retain stay id: ', len(analysis_cohort_for_eicu))

check_eicu = eicu_annoted[eicu_annoted['patientunitstayid'].isin(analysis_cohort_for_eicu)].reset_index(drop=True)
eicu_labeled = optimized_shock_labeler(check_eicu, 'eicu')

In [ ]:
# mimic_labeled.drop('Unnamed: 0', axis = 1).to_csv('check_point_mimic_labeled.csv.gz', compression = 'gzip', index=False)
# eicu_labeled.drop('Unnamed: 0', axis = 1).to_csv('check_point_eicu_labeled.csv.gz', compression = 'gzip', index=False)

In [2]:
import pandas as pd
import numpy as np

mimic_labeled = pd.read_csv('check_point_mimic_labeled.csv.gz', compression = 'gzip')
eicu_labeled = pd.read_csv('check_point_eicu_labeled.csv.gz', compression = 'gzip')

In [3]:
mimic_labeled['PaO2/FiO2'] = mimic_labeled['PaO2'] / ((mimic_labeled['FIO2 (%)']+ 0.00000001)/100)
eicu_labeled['PaO2/FiO2'] = eicu_labeled['PaO2'] / ((eicu_labeled['FIO2 (%)'] + 0.00000001)/100)

In [5]:
number(mimic_labeled, 'mimic')
number(eicu_labeled, 'eicu')

number of subject : 3218
number of stay : 3291
number of subject : 5411
number of stay : 5578


In [4]:
import Case
from imp import reload
reload(Case)
from tqdm import tqdm

In [5]:
invalid_columns_mimic, _ = Case.find_invalid_columns(mimic_labeled)
invalid_columns_eicu, _ = Case.find_invalid_columns(eicu_labeled)

m_labeled = Case.replace_inf_with_previous(mimic_labeled, invalid_columns_mimic)
e_labeled = Case.replace_inf_with_previous(eicu_labeled, invalid_columns_eicu)

In [6]:
reload(Case)
m_case1_case2, m_case3_case4 = Case.Case_definetion(m_labeled, 'mimic')
e_case1_case2, e_case3_case4 = Case.Case_definetion(e_labeled, 'eicu')

|Start MIMIC-IV process|
Extract Case 1, Case 2


100%|██████████| 3291/3291 [00:12<00:00, 261.97it/s]


--------------
Extract Case 3, Case 4


100%|██████████| 639/639 [00:12<00:00, 52.51it/s]


|Start eICU-CRD process|
Extract Case 1, Case 2


100%|██████████| 5563/5563 [00:11<00:00, 469.29it/s]


--------------
Extract Case 3, Case 4


100%|██████████| 708/708 [00:14<00:00, 49.75it/s]


In [7]:
m_case3_case4.Case.value_counts()

4.0    19344
3.0    14957
Name: Case, dtype: int64

In [8]:
m_case1_case2['INDEX'] = 'CASE1_CASE2_DF'
m_case3_case4['INDEX'] = 'CASE3_CASE4_DF'

e_case1_case2['INDEX'] = 'CASE1_CASE2_DF'
e_case3_case4['INDEX'] = 'CASE3_CASE4_DF'

In [9]:
mimic_analysis = pd.concat([m_case1_case2, m_case3_case4], axis = 0).reset_index(drop=True)
eicu_analysis = pd.concat([e_case1_case2, e_case3_case4], axis = 0).reset_index(drop=True)

In [10]:
mimic_ethnicity = {'AMERICAN INDIAN/ALASKA NATIVE': 0, 'ASIAN': 1,
 'ASIAN - ASIAN INDIAN': 2, 'ASIAN - CHINESE': 3,
 'ASIAN - KOREAN': 4, 'ASIAN - SOUTH EAST ASIAN': 5,
 'BLACK/AFRICAN': 6, 'BLACK/AFRICAN AMERICAN': 7,
 'BLACK/CAPE VERDEAN': 8, 'BLACK/CARIBBEAN ISLAND': 9,
 'HISPANIC OR LATINO': 10, 'HISPANIC/LATINO - CENTRAL AMERICAN': 11,
 'HISPANIC/LATINO - COLUMBIAN': 12, 'HISPANIC/LATINO - CUBAN': 13,
 'HISPANIC/LATINO - DOMINICAN': 14, 'HISPANIC/LATINO - GUATEMALAN': 15,
 'HISPANIC/LATINO - HONDURAN': 16, 'HISPANIC/LATINO - MEXICAN': 17,
 'HISPANIC/LATINO - PUERTO RICAN': 18, 'HISPANIC/LATINO - SALVADORAN': 19, 
 'MULTIPLE RACE/ETHNICITY': 20, 'NATIVE HAWAIIAN OR OTHER PACIFIC ISLANDER': 21, 
 'OTHER': 22, 'PATIENT DECLINED TO ANSWER': 23, 'PORTUGUESE': 24, 'SOUTH AMERICAN': 25,
 'UNABLE TO OBTAIN': 26, 'UNKNOWN': 27, 'WHITE': 28, 'WHITE - BRAZILIAN': 29,
 'WHITE - EASTERN EUROPEAN': 30, 'WHITE - OTHER EUROPEAN': 31, 'WHITE - RUSSIAN': 32}

eicu_ethnicity = {'0': 0, 'African American': 1, 'Asian': 2, 
                  'Caucasian': 3, 'Hispanic': 4, 'Native American': 5, 'Other/Unknown': 6}

new_mimic_ethnicity = {v:k for k,v in mimic_ethnicity.items()}
new_eicu_ethnicity = {v:k for k,v in eicu_ethnicity.items()}

In [11]:
mimic_analysis['ethnicity'] = mimic_analysis['ethnicity'].map(new_mimic_ethnicity)
eicu_analysis['ethnicity'] = eicu_analysis['ethnicity'].map(new_eicu_ethnicity)

In [12]:
A = {'AMERICAN INDIAN/ALASKA NATIVE': 1, 'ASIAN': 2, 'ASIAN - ASIAN INDIAN': 2, 
 'ASIAN - CHINESE': 2, 'ASIAN - KOREAN': 2, 'ASIAN - SOUTH EAST ASIAN': 2, 
 'BLACK/AFRICAN': 5, 'BLACK/AFRICAN AMERICAN': 5, 'BLACK/CAPE VERDEAN': 6, 
 'BLACK/CARIBBEAN ISLAND': 6, 'HISPANIC OR LATINO': 4, 'HISPANIC/LATINO - CENTRAL AMERICAN': 4, 
 'HISPANIC/LATINO - COLUMBIAN': 4, 'HISPANIC/LATINO - CUBAN': 4, 'HISPANIC/LATINO - DOMINICAN': 4, 
 'HISPANIC/LATINO - GUATEMALAN': 4, 'HISPANIC/LATINO - HONDURAN': 4, 'HISPANIC/LATINO - MEXICAN': 4, 
 'HISPANIC/LATINO - PUERTO RICAN': 4, 'HISPANIC/LATINO - SALVADORAN': 4, 'MULTIPLE RACE/ETHNICITY': 6, 
 'NATIVE HAWAIIAN OR OTHER PACIFIC ISLANDER': 6, 'OTHER': 6, 'PATIENT DECLINED TO ANSWER': 6, 'PORTUGUESE': 4, 
 'SOUTH AMERICAN': 5, 'UNABLE TO OBTAIN': 6, 'UNKNOWN': 6, 'WHITE': 3, 'WHITE - BRAZILIAN': 3, 
 'WHITE - EASTERN EUROPEAN': 3, 'WHITE - OTHER EUROPEAN': 3, 'WHITE - RUSSIAN': 3}

B = {'0': 6, 'African American': 1, 'Asian': 2, 'Caucasian': 3, 'Hispanic': 4, 'Native American': 5, 'Other/Unknown': 6}


mimic_analysis['ethnicity'] = mimic_analysis['ethnicity'].map(A)
eicu_analysis['ethnicity'] = eicu_analysis['ethnicity'].map(B)

In [15]:
mimic_analysis.to_csv('mimic_analysis(new_version0308).csv.gz', compression = 'gzip', index=False)
eicu_analysis.to_csv('eicu_analysis(new_version0308).csv.gz', compression = 'gzip', index=False)

In [16]:
number(mimic_analysis, 'mimic')
number(eicu_analysis, 'eicu')

NameError: name 'number' is not defined

In [192]:
mimic_analysis_past = pd.read_csv('mimic_analysis.csv.gz', compression = 'gzip')

C:\Users\DAHS\AppData\Local\Temp\ipykernel_23712\3583358883.py:1: DtypeWarning: Columns (225) have mixed types. Specify dtype option on import or set low_memory=False.
  mimic_analysis_past = pd.read_csv('mimic_analysis.csv.gz', compression = 'gzip')


In [195]:
mimic_analysis_past.Case.value_counts()

1.0      522948
4.0       19652
3.0       10382
2.0        5640
1.0        4096
3.0        1636
4.0        1551
event       705
Name: Case, dtype: int64

In [193]:
mimic_analysis_past.Case.value_counts()

1.0      522948
4.0       19652
3.0       10382
2.0        5640
1.0        4096
3.0        1636
4.0        1551
event       705
Name: Case, dtype: int64

In [184]:
eicu_analysis.INDEX.value_counts()

CASE1_CASE2_DF    929197
CASE3_CASE4_DF     30195
Name: INDEX, dtype: int64